In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_html("./test_table.html", encoding="UTF-8")

In [ ]:
## Надо нормализироват
## Это ТИП таблицы 2 - просто 

df[0].head()

In [ ]:
df2 = pd.read_html("./14_Таблицы__Волгоградская_область__.html", encoding="UTF-8")

In [ ]:
# ТИП ТАБЛИЦЫ

type_of_tb = 0 # 0 - тип где все года в одной таблице, 1 - тип где года в разных таблицах

# Какая размерность у столбца бюджетов

type_of_bg = 0 # 0 - тип где столбец описан в руб. 1 - мера столбца в тыс. руб

# Задаем константы для ИД таблицы

NAMES_ID = 0
CSR = []
VR = 0
ID_2023 = 0
ID_2024 = 0
ID_2025 = 0

# ШАГ 1 - Определяем заголовки, удаляем ненужные таблицы

In [ ]:
# Определяем в каких таблицах есть заголовки

def myfunc(x):
    try:
        return int(x)
    except:
        return x

title_index = []

for ind, i in enumerate(df2):
    flag = False
    for j, k in i.iloc[0:1].iterrows():
        new_arr = k.apply(lambda x: str(type(myfunc(x))))
        new_arr = set(new_arr)
    
    if len(new_arr) == 1:
        title_index.append(ind)

In [ ]:
print(title_index)

[0]


In [ ]:
# Определяем в каких таблицах есть вид расходов
# Если таблица с нужным заголовком не последняя, то получаем индекс следующей 
# за ней таблицей с заголовком и записываем его в переменную. Это для того, чтобы почистить хвост листа (возьмем данные от [need_title: bad_title])
# До нужной таблицы с заголовком так же не берем


# NAMES_ID = 0
# CSR = 0
# VR = 0
# ID_2023 = 0
# ID_2024 = 0
# ID_2025 = 0

import re
pattern = r'ВИД\w* РАСХОД\w*'

filter_title_index = []

for m in title_index:
    filter_title_index.append({m: False})

    for i, k in df2[m].iloc[:3].iterrows():
        for j in range(len(k)):
            if k[j] == "ВР" or re.findall(pattern, str(k[j]).upper()):
                del filter_title_index[-1]
                filter_title_index.append({m: True})


In [ ]:
filter_title_index

[{0: True}]

In [ ]:
true_arr = []

for i in filter_title_index:
    for x, y in i.items():
        if y:
            true_arr.append(x)

In [ ]:
true_arr

[0]

In [ ]:
# Определяем ТИП таблицы

if len(true_arr) > 1:
    type_of_tb = 1
else:
    type_of_tb = 0

print(type_of_tb)

0


In [ ]:
# Пробегаемся регулярками

pattern_csr = r'ЦЕЛЕВ\w* СТАТЬ\w*'
pattern_vr = r'ВИД\w* РАСХОД\w*'

def get_index(tb):

    for m in tb:
        prog = []
        
        for i, k in df2[m].iloc[:3].iterrows():
            for j in range(len(k)):
                
                if re.findall(r'НАИМЕНОВАНИЕ', str(k[j]).upper()):
                    global NAMES_ID
                    NAMES_ID = j

                if re.findall(r'ЦСР', str(k[j]).upper()) or re.findall(pattern_csr, str(k[j]).upper()):
                    prog = [*prog, j]
                    
                if k[j] == "ВР" or re.findall(pattern_vr, str(k[j]).upper()):
                    global VR 
                    VR = j

                if k[j] == "2023" or re.findall(r'2023', str(k[j]).upper()) or re.findall(r'СУММА', str(k[j]).upper()) or re.findall(r'ИТОГО', str(k[j]).upper()):
                    global ID_2023 
                    ID_2023 = j

                if re.findall(r'ТЫС\w* РУБ\w*', str(k[j]).upper()):
                    global type_of_bg 
                    type_of_bg = 1

                if k[j] == "2024" or re.findall(r'2024', str(k[j]).upper()):
                    global ID_2024 
                    ID_2024 = j
                
                if k[j] == "2025" or re.findall(r'2025', str(k[j]).upper()):
                    global ID_2025 
                    ID_2025 = j
        global CSR
        CSR.append(prog)
    

get_index(true_arr)

In [ ]:
print(NAMES_ID, CSR, VR, ID_2023, type_of_bg, ID_2024, ID_2025)

0 [[3]] 4 5 0 6 7


In [ ]:
# Чистим индексы в массиве таблиц
normalize_dfs = []
first_index = 0

for i, k in enumerate(filter_title_index):
    for j, q_bool in k.items():
        if q_bool:
            if len(filter_title_index) == i + 1:
                normalize_dfs.extend(df2[j:])
            else:
                for f, b_bool in filter_title_index[i+1].items():
                    normalize_dfs.extend(df2[j:f])

In [ ]:
len(normalize_dfs)

11

In [ ]:
len(df2)

11

#ШАГ 2 - Привожу к одному виду

In [ ]:
final_own = pd.DataFrame()

In [ ]:
# Определяем по типу как действовать (лучше проверить по len)

for i in normalize_dfs:
    final_df = pd.DataFrame()

    i = i.drop(0)
    # Отрабатываем тип 0 таблицы
    if len(i.columns) == ID_2023 + 3:
        final_df["Наименование"] = i[NAMES_ID]
        if len(CSR[0]) > 1:
            final_df['ЦСР'] = i.iloc[:, CSR[0]].astype(str).agg(''. join , axis= 1) 
        else:
            final_df["ЦСР"] = i[CSR[0]]
        final_df["ВР"] = i[VR]
        final_df["2023"] = i[ID_2023]
        final_df["2024"] = i[ID_2024]
        final_df["2025"] = i[ID_2025]


    # Отрабатываем тип 1 таблицы
    elif len(i.columns) == ID_2023 + 2:
        final_df["Наименование"] = i[NAMES_ID]
        if len(CSR[1]) > 1:
            final_df['ЦСР'] = i.iloc[:, CSR[1]].astype(str).agg(''. join , axis= 1) 
        else:
            final_df["ЦСР"] = i[CSR[1]]
        final_df["ВР"] = i[VR]
        final_df["2024"] = i[ID_2024]
        final_df["2025"] = i[ID_2025]

    elif len(i.columns) == ID_2023 + 1:
        final_df["Наименование"] = i[NAMES_ID]
        if len(CSR[1]) > 1:
            final_df['ЦСР'] = i.iloc[:, CSR[0]].astype(str).agg(''. join , axis= 1) 
        else:
            final_df["ЦСР"] = i[CSR[0]]
        final_df["ВР"] = i[VR]
        final_df["2023"] = i[ID_2023]

        
    
    #Обновляем df
    final_own = pd.concat([final_own, final_df])

In [ ]:
def delete_abrakadabra(x):
    try:
        x = x.strip()
        arr = []
        for i in x:
            if i.isdigit():
                arr.append(i)
        x = "".join(arr)
        return x
    except:
        return np.nan

In [ ]:
final_own['ЦСР'] = final_own['ЦСР'].apply(lambda x: delete_abrakadabra(x))

In [ ]:
final_own = final_own.drop([0, 1])

In [ ]:
final_own.reset_index(drop= True , inplace= True )

In [ ]:
final_own

,Наименование,ЦСР,ВР,2023,2024,2025
0,ВОЛГОГРАДСКАЯ ОБЛАСТНАЯ ДУМА,NaN,NaN,"331 966,8","307 870,4","306 206,6"
1,ОБЩЕГОСУДАРСТВЕННЫЕ ВОПРОСЫ,NaN,NaN,"331 966,8","307 870,4","306 206,6"
2,Функционирование законодательных (представител...,NaN,NaN,"273 676,8","252 680,4","253 016,6"
3,Непрограммные направления обеспечения деятельн...,900,NaN,"273 676,8","252 680,4","253 016,6"
4,Расходы на выплаты персоналу в целях обеспечен...,900,100,"240 678,8","223 221,0","223 221,0"
...,...,...,...,...,...,...
1282,ОБРАЗОВАНИЕ,NaN,NaN,2626,947,915
1283,"Профессиональная подготовка, переподготовка и ...",NaN,NaN,2626,947,915
1284,Непрограммные расходы государственных органов ...,990,NaN,2626,947,915
1285,"Предоставление субсидий бюджетным, автономным ...",990,600.0,2626,947,915


In [ ]:
final_own.to_html("test2.html")